In [8]:
from neo4j import GraphDatabase
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
import getpass
import os
import os


graph = Neo4jGraph(
    url="neo4j+s://28e5b287.databases.neo4j.io",
    username="neo4j",
    password=
)

name = 'OPENAI_API_KEY'
secret = getpass.getpass("Enter the OpenAI API Key")
os.environ[name] = secret
print(f"Have successfully set up {name} as an environment variable.")

# Initialize connection to Neo4j database
uri = "neo4j+s://28e5b287.databases.neo4j.io" 
auth = 
driver = GraphDatabase.driver(uri, auth=auth)

# Setting up ChatGPT integration with Graph Database
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [55]:
few_shot_learning_prompt = '''
Consider the following example context string:

Node 3356 connects to Node 3549
Node 3356 connects to Node 7018
Node 6939 connects to Node 1836
Node 4826 connects to Node 6939

In this context string, the nodes of the graph are called Node, with a property of id, which is the number above.
Each Node has relationship "CONNECTS" to another node.

There are more nodes in the graph than provided

Given the above, generate a Cypher query to create a context string where:
1. Matches all nodes of type 'Node' with property 'id' that are connected via a 'CONNECTED' relationship.
2. For each matched node and relationship, construct a context string that describes the connection. Use the '+' operator for string concatenation. The context string should include the names of the nodes and the cost of the ROAD.

'''

In [56]:
# Use ChatGPT to generate a cypher query about the context of the 

import io
import sys
import re

# Redirect stdout (standard output) to capture printed text
backup = sys.stdout
sys.stdout = io.StringIO()

# Run the function
chain.run(few_shot_learning_prompt)


# # Capture the printed text and revert stdout
captured_output = sys.stdout.getvalue()
sys.stdout.close()
sys.stdout = backup

match = re.search(r"Full Context:(.*)", captured_output, re.DOTALL)
if match:
    content = match.group(1).strip()

# Remove ANSI escape codes
cleaned_content = re.sub(r'\x1b\[.*?m', '', content)

In [57]:
import ast

# The given input as a string
input_string = cleaned_content
# Convert the string to a list of dictionaries
data_list = ast.literal_eval(input_string.split('\n\n')[0])

# Extract the desired text
result_list = [item['context_string'] for item in data_list]
result_string = ", ".join(result_list)


In [58]:
formatted_string = result_string.replace(", ", ",\n")
print(formatted_string)

In [59]:
formatted_string

'3356 connects to 45896,\n3356 connects to 3549,\n3356 connects to 29608,\n3356 connects to 22652,\n3356 connects to 3257,\n3356 connects to 7018,\n3356 connects to 1836,\n6830 connects to 8758,\n6939 connects to 45896,\n6939 connects to 8758'

In [60]:
system_prompt = """
You are an assistant that helps to generate text to form nice and human understandable answers based.
The latest prompt contains the information, and you need to generate a human readable response based on the given information.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
Do not add any additional information that is not explicitly provided in the latest prompt.
I repeat, do not add any information that is not explicitly given.
"""
# context = 'A is connected to D with a cost of 100, A is connected to C with a cost of 50, A is connected to B with a cost of 50, B is connected to D with a cost of 40, C is connected to E with a cost of 80, C is connected to D with a cost of 40, D is connected to A with a cost of 69, D is connected to F with a cost of 80, D is connected to E with a cost of 30, E is connected to F with a cost of 40'
context = formatted_string

def generate_user_prompt(question, context):
   return f"""
   The question is {question}
   Answer the question by using the provided information:
   {context}
   """

In [61]:
import openai

def get_gpt4_response(system_prompt, prompt, context, api_key, model="gpt-4-1106-preview"):
    """
    Send a request to the OpenAI Chat API and get a response from the model.
    
    Args:
        prompt (str): The user's question.
        context (str): Contextual information to be passed to the model.
        api_key (str): Your OpenAI API key.
        model (str): The model version to use, default is "gpt-3.5-turbo".
    
    Returns:
        str: The model's response.
    """
    
    # Create the full prompt by combining the system prompt, context, and the user question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": generate_user_prompt(prompt, context)}
    ]
    
    # Use the OpenAI Python client to send the request
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages
    )
    
    # Extract the response text and return
    return response.choices[0].message['content']

# You will need to install the openai Python package to use this function and also have a valid API key.


In [62]:
# Define your API key (keep it secret!)

openai.api_key = 
API_KEY = 

In [63]:
# Example usage
question = "How many nodes does the graph have"
context_info = context
response = get_gpt4_response(system_prompt, question, context_info, API_KEY, model = "gpt-3.5-turbo")
print(response)

In [64]:
response

'Based on the provided information, it appears that there are a total of 11 nodes in the graph.'